In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import pandas as pd
from keras import backend as K
import random
seed = 5
random.seed(seed)

Using TensorFlow backend.


In [2]:
train_data = pd.read_csv('../input/train.csv').values
test_data = pd.read_csv('../input/test.csv').values


In [3]:
train_x = train_data[:,1:].reshape(train_data.shape[0],1,28,28)
test_x = test_data.reshape(test_data.shape[0],1,28,28)
X_train = train_x / 255.0
X_test = test_x /255.0
y_train = train_data[:,0]


In [4]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
y_train = lb.fit_transform(y_train)


In [5]:
X_train.shape

(42000, 1, 28, 28)

In [6]:
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(16, (3, 3), input_shape = (1,28,28), activation = 'relu',data_format='channels_first'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2),strides=2))

# Adding a second convolutional layer
classifier.add(Conv2D(16, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2),strides=2))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 32, activation = 'tanh'))
classifier.add(Dense(units = 10, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [7]:
classifier.fit(X_train, y_train,
          epochs=10,
          batch_size= 64, validation_split=0.2)


Train on 33600 samples, validate on 8400 samples
Epoch 1/10
33600/33600 [==============================] - 11s 330us/step - loss: 0.7538 - acc: 0.8430 - val_loss: 0.2773 - val_acc: 0.9400
Epoch 2/10
33600/33600 [==============================] - 11s 314us/step - loss: 0.2139 - acc: 0.9485 - val_loss: 0.1647 - val_acc: 0.9607
Epoch 3/10
33600/33600 [==============================] - 10s 312us/step - loss: 0.1448 - acc: 0.9615 - val_loss: 0.1256 - val_acc: 0.9664
Epoch 4/10
33600/33600 [==============================] - 11s 313us/step - loss: 0.1142 - acc: 0.9678 - val_loss: 0.1093 - val_acc: 0.9685
Epoch 5/10
33600/33600 [==============================] - 11s 313us/step - loss: 0.0942 - acc: 0.9734 - val_loss: 0.0965 - val_acc: 0.9727
Epoch 6/10
33600/33600 [==============================] - 10s 312us/step - loss: 0.0811 - acc: 0.9771 - val_loss: 0.0824 - val_acc: 0.9756
Epoch 7/10
33600/33600 [==============================] - 11s 313us/step - loss: 0.0703 - acc: 0.9808 - val_loss: 0.0

In [8]:
a= classifier.predict(X_test)
a

array([[1.9192696e-05, 8.4519386e-05, 8.6669785e-01, ..., 3.0311942e-04,
        7.0929527e-06, 2.0146370e-05],
       [1.9708657e-01, 3.1590462e-06, 1.5452504e-04, ..., 7.5966120e-05,
        1.0728836e-05, 4.7653913e-05],
       [1.5884638e-05, 1.7881393e-06, 9.9539757e-06, ..., 1.7040968e-04,
        7.0038438e-04, 4.6153772e-01],
       ...,
       [1.3321638e-05, 5.2571297e-05, 1.4090538e-04, ..., 1.0237098e-04,
        1.6656518e-04, 1.5357137e-04],
       [1.4424324e-05, 7.6293945e-06, 6.4074993e-06, ..., 8.0227852e-05,
        5.5730343e-05, 9.2280358e-02],
       [7.1823597e-06, 1.5944242e-05, 1.1204204e-01, ..., 8.1360340e-05,
        2.2053719e-06, 1.0579824e-05]], dtype=float32)

In [9]:
y = [i.argmax() for i in a]

In [10]:
ans = pd.DataFrame(columns=['ImageId','Label'])
ans

,ImageId,Label


In [11]:
len(y)

28000

In [12]:
ans['ImageId'] = pd.Series(range(1,28001))

In [13]:
ans['Label'] = pd.Series(y)

In [14]:
ans

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [15]:
ans.to_csv('submission.csv',index=False)